In [ ]:
#import healpy as hp
from functools import partial
import astropy_healpix.healpy as ahp
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy_healpix import HEALPix
from hpmoc.healpy import healpy as hhp
from hpmoc.utils import N_X_OFFSET, N_Y_OFFSET, FONT_SIZE, wcs2ang
from hpmoc.plotters import outline_effect
from hpmoc.plot import (
    get_wcs, get_frame_class, _WCS_HEADERS, MIN_GRAT,
    DELTA_PARALLELS, DELTA_MERIDIANS, _DELTA_HIGHRES,
    get_ticks, ra_exclusions, dec_exclusions,
    _RA_EXCLUSIONS, _DEC_EXCLUSIONS, _PROJ_SETTINGS
)
from hpmoc.points import PointsTuple, Rgba
from astropy.units import Unit, deg
from matplotlib import pyplot as plt, cm
import numpy as np
import hpmoc
from hpmoc.partial import PartialUniqSkymap
from reproject import reproject_from_healpix, reproject_to_healpix
import astropy as ap
from astropy.units import deg, rad, Unit, Quantity
from astropy.wcs import WCS
from astropy.io import fits
from astropy.visualization.wcsaxes import WCSAxes, WCSAxesSubplot
from astropy.visualization.wcsaxes.frame import EllipticalFrame, RectangularFrame
from astropy.coordinates import FK5
#https://docs.astropy.org/en/stable/visualization/wcsaxes/generic_transforms.html
from matplotlib.transforms import Affine2D
from matplotlib.projections import projection_registry
import matplotlib.pyplot as plt
from matplotlib.transforms import ScaledTranslation

In [ ]:
SKYMAP = "../tests/data/S191216ap.fits.gz"
hdu_head, hdu_ligo = fits.open(SKYMAP)[:2]
m = PartialUniqSkymap.read(SKYMAP, strategy='ligo')
mo = PartialUniqSkymap.read("../tests/data/S191216ap.multiorder.fits", strategy='ligo')
# https://github.com/lpsinger/ligo.skymap/blob/main/ligo/skymap/plot/allsky.py#L791
target_header = fits.Header.fromstring("""
NAXIS   =                    2
NAXIS1  =                  480
NAXIS2  =                  240
CTYPE1  = 'RA---MOL'
CRPIX1  =                240.5
CRVAL1  =                180.0
CDELT1  =               -0.675
CUNIT1  = 'deg     '
CTYPE2  = 'DEC--MOL'
CRPIX2  =                120.5
CRVAL2  =                  0.0
CDELT2  =                0.675
CUNIT2  = 'deg     '
COORDSYS= 'icrs    '
""", sep='\n')
target_header['CDELT1'] = -np.sqrt(8)/np.pi*360/target_header['NAXIS1']
target_header['CDELT2'] = np.sqrt(8)/np.pi*180/target_header['NAXIS2']
w = WCS(target_header)
wh = WCS(hdu_ligo.header)
pts = PointsTuple(
    points=[
        (271., 31., 2., 'One'),
        (355., -44., 3.),
        (15., -62., 5.),
        (16., -60., 3.),
        (10., -30., 10.),
    ],
    rgba=Rgba(0, 1, 0, 0.3),
    marker='x',
    label='Test Points',
)
mo.point_sources = [pts]
cr, levels, norm = mo.quantiles([0.1, 1])
mr = m.render(w)
mor = mo.render(w)
array2, footprint2 = reproject_from_healpix("../tests/data/S191216ap.fits.gz",
                                            target_header, nested=True)

In [ ]:
(5*Unit('hourangle')+34.5*Unit('arcmin')).to('deg')

In [ ]:
_PROJ_SETTINGS['allsky']

In [ ]:
ax = m.plot(projection='SIN', rot=(60, 15, 0))

In [ ]:
def ticks(include, exclude, ticks, delta, transform):
    import numpy as np

    bins = np.repeat(ticks, 2).reshape((-1, 2))
    bins += np.array([-1, 1]) * delta
    bins = bins.ravel()
    ihist, _ = np.histogram(transform(include, delta), bins=bins)
    ehist, _ = np.histogram(transform(exclude, delta), bins=bins)
    matches = (ihist != 0) & (ehist == 0)
    return ticks[matches[::2]]

In [ ]:
type(w.pixel_to_world([1, 2], [2, 3]))

In [ ]:
pts_x, pts_y = w.world_to_pixel(SkyCoord(*[*zip(*((r, d) for r, d, *_ in pts.points))]*deg, frame='icrs'))

In [ ]:
np.array(pts.points)[include]

In [ ]:
include=(pts_x < w.pixel_shape[0]) & (pts_x > 0) & (pts_y < w.pixel_shape[1]) & (pts_y > 0)

In [ ]:
pts.points

In [ ]:
w.world_to_pixel?

In [ ]:
w.world_to_pixel(SkyCoord([30, 31]*deg, [2, 3]*deg, frame='icrs'))

In [ ]:
ra_exclusions(ax)

In [ ]:
get_ticks(wcs2ang(ax.wcs)[1].value+3, ra_exclusions(ax)+3, np.arange(30, 390, 30),
      3, lambda x, d: x+d%360)

In [ ]:
get_ticks(wcs2ang(ax.wcs)[2].value, dec_exclusions(ax), np.arange(-75, 90, 15),
      2, lambda x, _: x)

In [ ]:
ticks(wcs2ang(w)[2].value, 

In [ ]:
np.abs(ax.wcs.to_header()['CDELT1'])

In [ ]:
w.to_header()['CDELT1']

In [ ]:
ax = mo.plot()
#co_ra, co_dec = ax.coords
#co_ra.set_major_formatter("dd:mm")

In [ ]:
ELLIPTICAL_EDGE_EXCLUSION = 50

def _parametric_ra_exclusion(ra_height, w):
    import numpy as np

    return w.pixel_to_world(
        *np.meshgrid(*map(np.arange, w.pixel_shape), sparse=True)
    ).icrs.ra.value[
        ra_height(w),
        np.concatenate((
            np.arange(int(w.pixel_shape[0])//ELLIPTICAL_EDGE_EXCLUSION),
            -np.arange(1, int(w.pixel_shape[0])//ELLIPTICAL_EDGE_EXCLUSION),
        ))
    ]


def _dec_exclusion(w):
    import numpy as np
    
    dec = w.pixel_to_world(
            *np.meshgrid(*map(np.arange, w.pixel_shape), sparse=True)
        ).icrs.dec.value[
        np.concatenate((
            np.arange(int(w.pixel_shape[1])//ELLIPTICAL_EDGE_EXCLUSION),
            np.arange(1, int(w.pixel_shape[1])//ELLIPTICAL_EDGE_EXCLUSION),
        ))
    ]
    return dec[~np.isnan(dec)]


RA_EXCLUSIONS = {
    EllipticalFrame: partial(_parametric_ra_exclusion,
                             lambda w: int(w.pixel_shape[1])//2),
    object: partial(_parametric_ra_exclusion, lambda _: -1,),
}
DEC_EXCLUSIONS = {
    object: _dec_exclusion,
}


def _exclusions(registry, ax):
    w = ax.wcs
    frame_class = ax.coords.frame.__class__
    for c in frame_class.mro():
        if c in registry:
            return registry[c](w)
        

ra_exclusions = partial(_exclusions, RA_EXCLUSIONS)
ra_exclusions.__doc__ = "Get RA values to exclude from ticks."
dec_exclusions = partial(_exclusions, DEC_EXCLUSIONS)
dec_exclusions.__doc__ = "Get declination values to exclude from ticks."

In [ ]:
dec_exclusions(ax)

In [ ]:
ra_exclusions(w, EllipticalFrame)

In [ ]:
(lambda x: x[~np.isnan(x)])(
    w.pixel_to_world(
            *np.meshgrid(*map(np.arange, w.pixel_shape), sparse=True)
        ).icrs.dec.value[
        np.concatenate((
            np.arange(int(w.pixel_shape[1])//ELLIPTICAL_EDGE_EXCLUSION),
            np.arange(1, int(w.pixel_shape[1])//ELLIPTICAL_EDGE_EXCLUSION),
        ))
    ]
)

In [ ]:
RA_EXCLUSIONS[EllipticalFrame](w)

In [ ]:
np.concatenate((
                np.arange(int(ax.wcs.pixel_shape[0])//ELLIPTICAL_EDGE_EXCLUSION),
                -np.arange(1, int(ax.wcs.pixel_shape[0])//ELLIPTICAL_EDGE_EXCLUSION),
            ))

In [ ]:
def ra_transform(

In [ ]:
EllipticalFrame.mro()

In [ ]:
np.array([True, False]) & np.array([True, True])

In [ ]:
DELTA_MERIDIANS

In [ ]:
hpmoc.plotters.mollview(mo)
plt.show()

In [ ]:
EllipticalFrame.spine_names

In [ ]:
mo.mollview()
plt.show()

In [ ]:
ax2 = mo.plot(cr=[0.9, 0.99],
              cr_kwargs={'colors': 'blue'},
              projection="PAR",
              #hdelta=1,
              #vdelta=1,
              cr_filled=True,
              )
              # cr_format=lambda q, v: f"{100*q:.0f}% CR")
ax2.legend()

In [ ]:
mo.point_sources = []
ax2 = mo.plot(cr=[0.9],
              cmap='plasma',
              sigmas=[],
              cr_kwargs={'colors': 'blue'},
              projection="ZEA",
              #hdelta=4/np.pi,
              #vdelta=4/np.pi,
              frame_class=EllipticalFrame,
              rot=(315, 90, 0),
              )
              # cr_format=lambda q, v: f"{100*q:.0f}% CR")
#ax2.legend()
ax2.set_title("LVC/IceCube")

In [ ]:
ax2.set_title("

In [ ]:
mo.plot(cmap=None, cr=[0.5, 0.9])

In [ ]:
import astropy_healpix.healpy as ahp
import healpy as hp
from hpmoc.healpy import healpy as hhp

In [ ]:
hp.ang2vec([1, 2], [3, 4], lonlat=False).shape == (3, 2)

In [ ]:
ax2.contour?

In [ ]:
ax1 = mo.plot()
ax1.contour(mo.render(ax1.wcs), transform=ax1.get_transform(ax1.wcs),
            levels=mo.quantiles([0, 0.1])[1][1:], colors='blue')

In [ ]:
levels

In [ ]:
mocr.plot()

In [ ]:
mo.plot(width=720, height=360)

In [ ]:
fig = plt.figure(dpi=200)
mo.plot(fig=fig)

- [Some examples of setting the coordinate formatter](https://wcsaxes.readthedocs.io/en/latest/ticks_labels_grid.html)
- [Seemingly same site here](https://docs.astropy.org/en/stable/visualization/wcsaxes/ticks_labels_grid.html)
- [WCS Example FITS files](https://www.atnf.csiro.au/people/mcalabre/WCS/example_data.html)